### Import Data

Need to take the .las file containing the point cloud and sample 1000 points from it. Sampling should be done uniformly.


Data from:
https://www.lpi.usra.edu/publications/books/barringer_crater_guidebook/LiDAR/

Map of area:
http://opentopo.sdsc.edu/lidarDataset?opentopoID=OTLAS.112011.26912.3



In [ ]:
# !pip install laspy
pip install dionysus

In [ ]:
from laspy.file import File
import numpy as np

# Get file
las_file = "points.las"
# Read file
inFile = File(las_file, mode='r')
data = inFile.points
# inFile.close() # this must be placed at the end after all calculations have been made using data

In [ ]:
head = inFile.header
max_x, max_y, max_z = head.max
min_x, min_y, min_z = head.min

range_x = (max_x-min_x)
range_y = (max_y-min_y)
range_z = (max_z-min_z)

print(max_x,max_y,max_z)
print(min_x,min_y,min_z)

In [ ]:
import random

n = len(data)
num_points_sampled = 1000
xs = np.zeros(num_points_sampled)
ys = np.zeros(num_points_sampled)
zs = np.zeros(num_points_sampled)

for i in range(3):
    # Get list of random indices between 0 and len(data)
    rand_indices = random.sample(range(n+1), num_points_sampled)
    for c, idx in enumerate(rand_indices):
        xs[c] = (data[idx][0][0]/100 - min_x)/(max_x-min_x)
        ys[c] = (data[idx][0][1]/100 - min_y)/(max_y-min_y)
        zs[c] = (data[idx][0][2]/100 - min_z)/(max_z-min_z)
    np.save('sample_points_{}'.format(i), np.stack((xs,ys,zs), axis=-1))

In [ ]:
# point_cloud_1 = np.stack((xs,ys,zs), axis=-1)
# np.save('pc1.npy', point_cloud_1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs, c='r', marker='.')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

In [ ]:
# Code by Mark Leadingham
def better_plots(dgms,deg):
    new_death=0
    for degree, dgm in enumerate(dgms):
        for point in dgm:
            if point.death != float('inf'):
                if point.death>new_death:
                    new_death=float(point.death)
    new_death=new_death*1.25
    #Plot the barcodes
#     fig=plt.figure()
    
    patches = []
    iterate=0
    for degree, dgm in enumerate(dgms):
        if degree>=deg-1:
            break
        fig=plt.figure(degree)
        for point in dgm:
            if float(point.death) != float('inf'): 
                plt.plot([point.birth,point.death], [iterate,iterate])
            else:
                plt.plot([point.birth, new_death], [iterate,iterate])
#                 arrow = mpatches.Arrow((iterate, point.birth), (iterate, point.death))
#                 patches.append(arrow)
            iterate+=1
        plt.title('Degree: '+str(degree))
        plt.xlabel('Parameter, r')
        plt.show()
    
    for degree, dgm in enumerate(dgms):
        if degree>=deg-1:
            break
        fig2=plt.figure(degree)
        for point in dgm:
            #if degree==2:
                #print(point.birth, point.death)
            if float(point.death) != float('inf'): 
                plt.scatter(point.birth,point.death, figure=fig2)
            else:
                plt.scatter(point.birth, new_death, figure=fig2)
        maxe=new_death
            
        plt.plot([0,maxe],[0,maxe], figure=fig2)
        plt.title('Degree: '+str(degree))
        plt.show()

In [ ]:
import dionysus as dio
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

compressed_M = squareform(pdist(point_cloud_1,'euclidean'))
fil_rips = d.fill_rips(compressed_M, 3, 3)
fil_cech = d.fill_cech(compressed_M, 3, 3)
h = d.homology_persistence(fil_rips)
dgms = d.init_diagrams(h,fil_rips)

better_plots(dgms,3)